In [1]:
import numpy as np
import pandas as pd


# random_tweet_path = '/Users/jmooney/Downloads/tweets_random_all.xlsx'
# isis_tweet_path = '/Users/jmooney/Downloads/tweets_isis_all.xlsx'

# random_tweets = pd.read_excel(random_tweet_path)
# isis_tweets = pd.read_excel(isis_tweet_path)

In [4]:
tweet_path = '/Users/jmooney/Downloads/tweets.csv'
random_tweet_path = '/Users/jmooney/Documents/tweets_random_all.csv'
isis_tweets = pd.read_csv(tweet_path)

random_tweets = pd.read_csv(random_tweet_path)
random_tweets['is_isis'] = 0
isis_tweets['is_isis'] = 1
isis_tweets= isis_tweets.rename(columns={'tweets':'content'})
isis_tweets = isis_tweets[['content', 'is_isis']]
random_tweets = random_tweets[['content', 'is_isis']]
all_tweets = isis_tweets.append(random_tweets)

In [5]:
isis_tweets.head()

,content,is_isis
0,ENGLISH TRANSLATION: 'A MESSAGE TO THE TRUTHFU...,1
1,ENGLISH TRANSLATION: SHEIKH FATIH AL JAWLANI '...,1
2,ENGLISH TRANSLATION: FIRST AUDIO MEETING WITH ...,1
3,ENGLISH TRANSLATION: SHEIKH NASIR AL WUHAYSHI ...,1
4,ENGLISH TRANSLATION: AQAP: 'RESPONSE TO SHEIKH...,1


In [6]:
random_tweets.head()

,content,is_isis
0,We're #hiring! Read about our latest #job open...,0
1,No sir. You won because of Comey\u2019s hubris...,0
2,@theintercept @JulianAssange Equadorians shoul...,0
3,@GMS_WWE @ProWrestlingMag @WWERollins Agreed. ...,0
4,@ArvindKejriwal This LG is a centre's puppet. ...,0


In [9]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf
# import tensorflow_text as text
# from official.nlp import optimization
# import tensorflow_hub as hub

BUFFER_SIZE = 1000
BATCH_SIZE = 32

X = all_tweets[['content']]
y = all_tweets[['is_isis']]
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify=y, 
                                                    test_size=0.25)

# TOKENIZE

train_text = X_train['content'].to_numpy()
tok = Tokenizer(oov_token='<unk>')
tok.fit_on_texts(train_text)
tok.word_index['<pad>'] = 0
tok.index_word[0] = '<pad>'

train_seqs = tok.texts_to_sequences(train_text)
train_seqs = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')

train_labels = y_train.to_numpy().flatten()

valid_text = X_test['content'].to_numpy()
valid_seqs = tok.texts_to_sequences(valid_text)
valid_seqs = tf.keras.preprocessing.sequence.pad_sequences(valid_seqs, padding='post')

valid_labels = y_test.to_numpy().flatten()

# CONVERT TO TF DATASETS

train_ds = tf.data.Dataset.from_tensor_slices((train_seqs,train_labels))
valid_ds = tf.data.Dataset.from_tensor_slices((valid_seqs,valid_labels))

train_ds = train_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
valid_ds = valid_ds.batch(BATCH_SIZE)

# PREFETCH

train_ds = train_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
valid_ds = valid_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


In [22]:
import tensorflow_hub as hub

bert_model = hub.KerasLayer(tfhub_handle_encoder)
bert_results = bert_model(text_preprocessed)

print(f'Loaded BERT: {tfhub_handle_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

ModuleNotFoundError: No module named 'tensorflow_hub'

In [ ]:
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
    return tf.keras.Model(text_input, net)

In [75]:
for text_batch, label_batch in train_ds.take(1):
    for i in range(3):
        print(f'Review: {text_batch.numpy()[i]}')
        label = label_batch.numpy()[i]
        print(f'Label : {label} ({class_names[label]})')

RuntimeError: __iter__() is only supported inside of tf.function or when eager execution is enabled.

In [ ]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)


In [16]:
! source venv/bin/activate

In [17]:
! pip install --upgrade pip


In [18]:
! pip3 install -q -U tensorflow-text

In [19]:
! pip3 install -q tf-models-official

In [31]:
! pip3 install "tensorflow>=2.5"
! pip3 install --upgrade tensorflow-hub

  Using cached tensorflow-2.5.0-cp36-cp36m-macosx_10_11_x86_64.whl (195.6 MB)
  Using cached tensorflow_estimator-2.5.0-py2.py3-none-any.whl (462 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached h5py-3.1.0-cp36-cp36m-macosx_10_9_x86_64.whl (2.9 MB)
  Using cached tensorboard-2.5.0-py3-none-any.whl (6.0 MB)
  Using cached numpy-1.19.5-cp36-cp36m-macosx_10_9_x86_64.whl (15.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Attempting uninstall: h5py
    Found existing installation:

In [36]:
! pip3 install --upgrade tensorflow_text

In [37]:
import os
import shutil

import tensorflow as tf
# import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

ModuleNotFoundError: No module named 'tensorflow_text'

In [23]:
!python3 -version

Unknown option: -e
usage: /usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/Resources/Python.app/Contents/MacOS/Python [option] ... [-c cmd | -m mod | file | -] [arg] ...
Try `python -h' for more information.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.11.161 requires botocore==1.7.19, but you have botocore 1.12.93 which is incompatible.
awscli 1.11.161 requires PyYAML<=3.12,>=3.10, but you have pyyaml 5.4.1 which is incompatible.
awscli 1.11.161 requires s3transfer<0.2.0,>=0.1.9, but you have s3transfer 0.2.0 which is incompatible.


In [20]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

ModuleNotFoundError: No module named 'tensorflow_hub'

In [8]:
! pip3 install -q -U tensorflow-text
! pip3 install -q tf-models-official

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
^C
ERROR: Operation cancelled by user
DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support


In [7]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

ModuleNotFoundError: No module named 'tensorflow_hub'